In [1]:
import pandas as pd
import requests
import plotly.graph_objects as go
import math
from scipy import stats
import string
import numpy as np
import time
from scipy.stats import zscore
import sys
import os

def lineuppull(team_id, season, opp=False, ps=False):
    term = "Opponent" if opp else "Team"
    s_type = "Playoffs" if ps else "Regular Season"
    
    wowy_url = "https://api.pbpstats.com/get-wowy-stats/nba"
    print(team_id)
    wowy_params = {
        "TeamId": team_id,
        "Season": season,
        "SeasonType": s_type,
        "Type": term
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0',
    }
    
    wowy_response = requests.get(wowy_url, params=wowy_params, headers=headers)
    wowy = wowy_response.json()
    combos = wowy["multi_row_table_data"]
    frame_length = len(combos)
    df = pd.DataFrame(combos, index=[0]*frame_length)
    return df

def get_filename(team_id, year, opp=False, ps=False):
    """Generate filename based on parameters"""
    filename = f"{team_id}"
    if opp:
        filename += "_vs"
    if ps:
        filename += "_ps"
    filename += ".csv"
    return filename

def pull_onoff(years, opp=False, ps=False):
    count = 0
    if ps == False:
        player_index = pd.read_csv('index_master.csv')
    else:
         player_index = pd.read_csv('index_master_ps.csv')
    player_index = player_index[player_index.team != 'TOT']
    player_index = player_index[player_index.year > 2000]
    player_index = player_index.drop_duplicates()
    all_frames = []
    
    for year in years:
        # Create year directory if it doesn't exist
        year_dir = f"data/{year}"
        os.makedirs(year_dir, exist_ok=True)
        
        season_index = player_index[player_index.year == year].reset_index(drop=True)
        season = f"{year-1}-{str(year)[-2:]}"
        
        frames = []
        fail_list = []
        
        for team_id in season_index.team_id.unique():
            # Generate filename for this team/year combination
            filename = get_filename(team_id, year, opp, ps)
            filepath = os.path.join(year_dir, filename)
            
            # Check if file already exists
            if os.path.exists(filepath):
                print(f"File already exists for team {team_id} in {year}, skipping...")
                # Optionally read existing file and add to frames
                existing_df = pd.read_csv(filepath)
                frames.append(existing_df)
                continue
            
            try:
                df = lineuppull(team_id, season, opp=opp, ps=ps)
                df = df.reset_index(drop=True)
                df['team_id'] = team_id
                df['year'] = year
                df['season'] = season
                df['team_vs'] = opp
                
                # Save individual team file
                df.to_csv(filepath, index=False)
                time.sleep(1)
                print(f"Saved data for team {team_id} in {year}")
                
                frames.append(df)
                count += 1
                
            except Exception as e:
                print(f"Error processing team {team_id} in {year}: {str(e)}")
                fail_list.append((team_id, year))
        
        if frames:
            year_frame = pd.concat(frames)
            all_frames.append(year_frame)
            print(f'Year {year} Completed')
        
    if fail_list:
        print("\nFailed to process the following team/year combinations:")
        for team, year in fail_list:
            print(f"Team: {team}, Year: {year}")
    
    return pd.concat(all_frames) if all_frames else pd.DataFrame()
        
#pull_onoff(years,opp=True,ps=True) 
#pull_onoff(years,opp=False,ps=True) 
years=[i for i in range(2025,2026)]
df = pull_onoff(years,opp=False,ps=False) 
df = pull_onoff(years,opp=True,ps=False) 

years=[i for i in range(2001,2025)]
#df = pull_onoff(years,opp=False,ps=True) 
#df = pull_onoff(years,opp=True,ps=True) 


File already exists for team 1610612747 in 2025, skipping...
File already exists for team 1610612749 in 2025, skipping...
File already exists for team 1610612738 in 2025, skipping...
File already exists for team 1610612753 in 2025, skipping...
File already exists for team 1610612737 in 2025, skipping...
File already exists for team 1610612745 in 2025, skipping...
File already exists for team 1610612751 in 2025, skipping...
File already exists for team 1610612742 in 2025, skipping...
File already exists for team 1610612756 in 2025, skipping...
File already exists for team 1610612765 in 2025, skipping...
File already exists for team 1610612746 in 2025, skipping...
File already exists for team 1610612743 in 2025, skipping...
File already exists for team 1610612755 in 2025, skipping...
File already exists for team 1610612750 in 2025, skipping...
File already exists for team 1610612739 in 2025, skipping...
File already exists for team 1610612741 in 2025, skipping...
File already exists for 

In [2]:
df

,EntityId,TeamId,Name,ShortName,RowId,TeamAbbreviation,SecondsPlayed,GamesPlayed,Minutes,PlusMinus,...,Defensive 3 Seconds Violations,BlockedCorner3,Arc3PctBlocked,Transition Take Fouls Drawn,3pt And 1 Free Throw Trips,BlockedLongMidRange,Transition Take Fouls,OffensiveGoaltends,3SecondViolations,Clear Path Fouls
0,1629629-1629637-1631108-1642261-1642355,1610612747,"Cam Reddish, Jaxson Hayes, Max Christie, Dalto...","Reddish, Hayes, Christie, Knecht, James",1629629-1629637-1631108-1642261-1642355,LAL,316.0,1,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1630559-1642261-1642355-203076-2544,1610612747,"Austin Reaves, Dalton Knecht, Bronny James, An...","Reaves, Knecht, James, Davis, James",1630559-1642261-1642355-203076-2544,LAL,161.0,1,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1629216-1629637-1630559-203076-2544,1610612747,"Gabe Vincent, Jaxson Hayes, Austin Reaves, Ant...","Vincent, Hayes, Reaves, Davis, James",1629216-1629637-1630559-203076-2544,LAL,85.0,1,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1629216-1629637-1630559-1631108-2544,1610612747,"Gabe Vincent, Jaxson Hayes, Austin Reaves, Max...","Vincent, Hayes, Reaves, Christie, James",1629216-1629637-1630559-1631108-2544,LAL,77.0,2,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1629216-1629637-1630559-1631108-203076,1610612747,"Gabe Vincent, Jaxson Hayes, Austin Reaves, Max...","Vincent, Hayes, Reaves, Christie, Davis",1629216-1629637-1630559-1631108-203076,LAL,82.0,2,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1629012-1631117-1641707-1642262-203903,1610612762,"Collin Sexton, Walker Kessler, Taylor Hendrick...","Sexton, Kessler, Hendricks, Williams, Clarkson",1629012-1631117-1641707-1642262-203903,UTA,17.0,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,1628374-1629012-1629234-1641707-1641718,1610612762,"Lauri Markkanen, Collin Sexton, Drew Eubanks, ...","Markkanen, Sexton, Eubanks, Hendricks, George",1628374-1629012-1629234-1641707-1641718,UTA,19.0,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,1628374-1628381-1629234-1641718-203903,1610612762,"Lauri Markkanen, John Collins, Drew Eubanks, K...","Markkanen, Collins, Eubanks, George, Clarkson",1628374-1628381-1629234-1641718-203903,UTA,199.0,1,3.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,1628374-1631117-1641707-201988-203903,1610612762,"Lauri Markkanen, Walker Kessler, Taylor Hendri...","Markkanen, Kessler, Hendricks, Mills, Clarkson",1628374-1631117-1641707-201988-203903,UTA,182.0,1,3.0,-12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
